In [1]:
#!/usr/bin/env python3
# Fine tune CodeT5 model on the FStar everest dataset.
from __future__ import absolute_import, division, print_function
import datetime
from typing import *
from loguru import logger
import multiprocessing
from tqdm import tqdm
import sys
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler
import torch
import numpy as np
import json
import random
import torch
import os
import argparse
from transformers.trainer_utils import EvalPrediction
from transformers import (
    AdamW, get_linear_schedule_with_warmup,
    BertConfig, BertForMaskedLM, BertTokenizer,
    GPT2Config, GPT2LMHeadModel, GPT2Tokenizer,
    OpenAIGPTConfig, OpenAIGPTLMHeadModel, OpenAIGPTTokenizer,
    RobertaConfig, RobertaModel, RobertaTokenizer,
    DistilBertConfig, DistilBertForMaskedLM, DistilBertTokenizer,
)
import wandb
import pandas as pd
from transformers import DataCollatorForLanguageModeling
from transformers import AutoModelForSequenceClassification
from transformers import Seq2SeqTrainer,AutoTokenizer, T5ForConditionalGeneration,EarlyStoppingCallback, Seq2SeqTrainingArguments, AdamW, ProgressCallback

# https://huggingface.co/transformers/v3.0.2/model_doc/t5.html#t5forconditionalgeneration
### Tutorial: https://huggingface.co/docs/transformers/main/tasks/masked_language_modeling

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"device: {device}")
assert device == "cuda:0"

model_name='kaiyuy/leandojo-lean4-sst-byt5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
max_model_length = tokenizer.model_max_length #for CodeT5 it is 512
# print(model)




device: cuda:0


In [3]:
outs_ids = model.generate(tokenizer("def id (n : Nat) : Nat := ", return_tensors="pt").to(device).input_ids,
    max_length=1024,
    num_beams=4,
    length_penalty=0.0,
    do_sample=False,
    num_return_sequences=4,
    early_stopping=False)
for ix in range(len(outs_ids)):
    out_str = tokenizer.decode(outs_ids[ix], skip_special_tokens=True)
    logger.info(f"{ix:3}> out_str: {out_str}")

2023-08-11 08:36:39.249 | INFO     | __main__:<module>:10 -   0> out_str: have id (n : Nat) : Nat := fun _ ↦ id
2023-08-11 08:36:39.255 | INFO     | __main__:<module>:10 -   1> out_str: exact id.def n ▸ go rfl
2023-08-11 08:36:39.264 | INFO     | __main__:<module>:10 -   2> out_str: have id (n : Nat) : Nat := fun n h => h.symm ▸ Nat.le_refl..
2023-08-11 08:36:39.271 | INFO     | __main__:<module>:10 -   3> out_str: have id (n : Nat) : Nat := fun _ h ↦ h


In [4]:
eos_token = tokenizer.eos_token
logger.info(f"model EOS token: {eos_token}")
logger.info(f"special tokens: {tokenizer.all_special_tokens}")
# #updating the tokenizer's vocalblary file with End of Statement <EOS> Special Token:
# # print("Tokenizer's original size:  ",len(tokenizer))
# special_tokens_dict = {'eos_token': '<EOS>'}
# num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
# # print('\n We have added', num_added_toks, 'token')
# model.resize_token_embeddings(len(tokenizer))
# # print(tokenizer.all_special_tokens)


2023-08-11 08:36:39.276 | INFO     | __main__:<module>:2 - model EOS token: </s>
2023-08-11 08:36:39.278 | INFO     | __main__:<module>:3 - special tokens: ['</s>', '<unk>', '<pad>', '<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>', '<extra_id_46>', '<extra_id_47>', '<extra_id_

In [5]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: bollu (microsoft-research-incubation). Use `wandb login --relogin` to force relogin


True

In [6]:
# def get_label_ids_sakina(target):
#     """
#     Why is it correct for the model to produce <EOS> at the end if we have too large a sentence?
#     Siddharth does not believe this implementation.
#     """
#     max_length=tokenizer.model_max_length
#     # to train model on End of statement token. Even When model truncates longer code, EOS remain to show model the end of the statement
#     # Tokenize the target text without padding to get the tokens
#     encoded_tokens = tokenizer.tokenize(target)
#     # Check if the total number of tokens is greater than max_length
#     if len(encoded_tokens) > max_length:
#         # If yes, truncate the tokens while preserving the "<EOS>" at the end
#         truncated_tokens = encoded_tokens[:max_length - 1] + [encoded_tokens[-1]]
#         # Convert the truncated tokens back to input_ids
#         input_ids = tokenizer.convert_tokens_to_ids(truncated_tokens)
#     else:
#         # If no truncation needed, keep the original tokens with padding
#         input_ids = tokenizer(target, max_length=max_length, padding="max_length", truncation=True).input_ids
#     # print(input_ids)
#     return input_ids


# def get_label_ids(target):
#     """
#     get  {'input_ids': [1, 11351, 4653, 29025, 719, 2616, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}
#     for input "foo bar baz quux larp"
#     """
#     input_ids = tokenizer(target, max_length=max_length, padding="max_length", truncation=True).input_ids
#     return input_ids

# if True: # testing
#     eos_encoded = tokenizer(tokenizer.eos_token)
#     logger.debug(f'eos token: {tokenizer.eos_token} | encoded {eos_encoded}')
#     logger.debug(f'tokenizer tokenize: {tokenizer.tokenize("foo bar baz quuxlajdasdsadlka")}')
#     logger.debug(f'tokenizer funcall: {tokenizer("foo bar baz quux")}')
#     logger.debug(f'convert_tokens_to_ids: {tokenizer.convert_tokens_to_ids(tokenizer.tokenize("foo bar baz quux larp"))}')
# # tokenize: string -> List[token=str]
# # convert_tokens_to_ids: List[token=str] -> List[int]
# # tokenizer(...) = convert_tokns_to_ids . tokenize + attention mask.
# # 
# if True: # testing
#     eos_encoded = tokenizer(tokenizer.eos_token)
#     logger.debug(f'eos token: {tokenizer.eos_token} | encoded {eos_encoded}')
#     logger.debug(f'tokenizer tokenize: {tokenizer.tokenize("foo bar baz quuxlajdasdsadlka")}')
#     logger.debug(f'tokenizer funcall: {tokenizer("foo bar baz quux")}')
#     logger.debug(f'convert_tokens_to_ids: {tokenizer.convert_tokens_to_ids(tokenizer.tokenize("foo bar baz quux larp"))}')

In [7]:
#maping the dataset into batches
Experiment = True # for the first experimental run to get the pipeline going
if Experiment and not os.path.exists("input.txt"):
    # Download Shakespeare
    %time
    !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    

In [8]:
import datasets
import tqdm
import json
import pathlib
import glob

defs = []

files = set()
for jpath in tqdm.tqdm(glob.glob("dataset/*.json")):
    contents = open(jpath, "r").read()
    if not contents: continue
    j = json.loads(contents)
    for jdefn in j["defs"]:
        filepath = pathlib.Path(jdefn["file_name"]).name 
        files.add(filepath)
        data = open(f"./raw_dataset/{filepath}").readlines()
        start_line = int(jdefn["start_line"])
        end_line = int(jdefn["end_line"])
        if start_line == 0: continue # start line is zero.
        data = "\n".join(data[start_line-1:end_line-1])
        if data:
            defs.append({"input": data + tokenizer.eos_token})
        
print(defs[:3])
print(f"#defs: {len(defs)}")
files = sorted(list(files))
print(f"grabbed output from #files: {len(files)}")

100%|███████████████████████████████████████████████████████████████████████████████████████████| 2699/2699 [00:21<00:00, 123.29it/s]

[{'input': '  = let open FStar.List.Tot in\n\n    let r = reveal r in\n\n    if length r = 0 then 0\n</s>'}, {'input': 'val lemma_disjoint_includes (i:rid) (j:rid) (k:rid)\n\n  :Lemma (requires  (disjoint i j /\\ includes j k))\n\n         (ensures   (disjoint i k))\n\n         (decreases (List.Tot.length (reveal k)))\n</s>'}, {'input': 'val lemma_includes_refl (i:rid)\n\n  :Lemma (includes i i)\n</s>'}]
#defs: 49728
grabbed output from #files: 2647


In [9]:
# https://github.com/huggingface/notebooks/blob/main/examples/language_modeling-tf.ipynb
### Tutorial: https://huggingface.co/docs/transformers/main/tasks/masked_language_modeling
def build_huggingface_dataset_from_list_of_defs(defs: List[Dict[str, Any]]) -> datasets.Dataset:
    dataset = datasets.Dataset.from_list(defs)
    dataset = dataset.map(lambda egs : tokenizer(egs["input"]), batched=True, num_proc=4)
    dataset = dataset.remove_columns(["input"]) # delete the key that is not a list of ints.
    # block_size = 128
    block_size = 128
    def group_texts(examples):
        # Concatenate all texts.
        concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()} # collapse everything into a single [int]
        # concatenated_examples = examples
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        # We drop the small remainder, though you could add padding instead if the model supports it
        # In this, as in all things, we advise you to follow your heart
        total_length = (total_length // block_size) * block_size
        # Split by chunks of max_len.
        result = {
            k: [v[i : i + block_size] for i in range(0, total_length, block_size)]
            for k, v in concatenated_examples.items()
        } # split stuff into new [int]
        result["labels"] = result["input_ids"].copy() # not sure why this is correct -_^, ah yes it's correct because we are trying to do next token prediction.
        return result
    dataset = dataset.map(group_texts, batched=True, batch_size=1000, num_proc=4)
    return dataset 

TOTAL_LEN = len(defs)
TRAIN_SPLIT_IX = int(TOTAL_LEN*0.8)
VALID_SPLIT_IX = int(TOTAL_LEN*0.9)
train_dataset = build_huggingface_dataset_from_list_of_defs(defs[:TRAIN_SPLIT_IX])
valid_dataset = build_huggingface_dataset_from_list_of_defs(defs[TRAIN_SPLIT_IX:VALID_SPLIT_IX])
test_dataset = build_huggingface_dataset_from_list_of_defs(defs[VALID_SPLIT_IX:])
logger.info(f"len train: {TRAIN_SPLIT_IX} | test: {VALID_SPLIT_IX - TRAIN_SPLIT_IX} | valid: {TOTAL_LEN-VALID_SPLIT_IX}")

Map (num_proc=4):   0%|          | 0/39782 [00:00<?, ? examples/s]

/home/t-sibhat/.local/lib/python3.9/site-packages/transformers/models/byt5/tokenization_byt5.py:149: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/home/t-sibhat/.local/lib/python3.9/site-packages/transformers/models/byt5/tokenization_byt5.py:149: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/home/t-sibhat/.local/lib/python3.9/site-packages/transformers/models/byt5/tokenization_byt5.py:149: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/home/t-sibhat/.local/lib/python3.9/site-packages/transformers/models/byt5/tokenization_byt5.py:149: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Map (num_proc=4):   0%|          | 0/39782 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4973 [00:00<?, ? examples/s]

/home/t-sibhat/.local/lib/python3.9/site-packages/transformers/models/byt5/tokenization_byt5.py:149: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/home/t-sibhat/.local/lib/python3.9/site-packages/transformers/models/byt5/tokenization_byt5.py:149: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/home/t-sibhat/.local/lib/python3.9/site-packages/transformers/models/byt5/tokenization_byt5.py:149: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/home/t-sibhat/.local/lib/python3.9/site-packages/transformers/models/byt5/tokenization_byt5.py:149: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Map (num_proc=4):   0%|          | 0/4973 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4973 [00:00<?, ? examples/s]

/home/t-sibhat/.local/lib/python3.9/site-packages/transformers/models/byt5/tokenization_byt5.py:149: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/home/t-sibhat/.local/lib/python3.9/site-packages/transformers/models/byt5/tokenization_byt5.py:149: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/home/t-sibhat/.local/lib/python3.9/site-packages/transformers/models/byt5/tokenization_byt5.py:149: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/home/t-sibhat/.local/lib/python3.9/site-packages/transformers/models/byt5/tokenization_byt5.py:149: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Map (num_proc=4):   0%|          | 0/4973 [00:00<?, ? examples/s]

2023-08-11 08:37:28.586 | INFO     | __main__:<module>:33 - len train: 39782 | test: 4973 | valid: 4973


In [10]:
def debug():
    train_eg = next(iter(train_dataset))
    print(f"train_eg keys: {train_eg.keys()}")
    print(f"train_eg len: {[len(train_eg[k]) for k in train_eg.keys()]}")
    print(f"train_eg vals: {[(k, train_eg[k][:1]) for k in train_eg.keys()]}")
    
    # train_eg_input = train_eg['input']
    # print(f"train_eg input: {type(train_eg_input)} | len : {len(train_eg_input)}")
    # print(f"input[0]: {train_eg_input[0][:90]}")
debug()

train_eg keys: dict_keys(['input_ids', 'attention_mask', 'labels'])
train_eg len: [128, 128, 128]
train_eg vals: [('input_ids', [35]), ('attention_mask', [1]), ('labels', [35])]


In [11]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="output_dir",
    learning_rate=1e-3, # should I use a much smaller learning rate?
    num_train_epochs=30,
    weight_decay=1e-2,
    logging_steps=1,
    evaluation_strategy='steps',
    eval_steps=10,
    per_device_train_batch_size=2, # can double?
    per_device_eval_batch_size=2 # can quadruple?
)
training_args = training_args.set_dataloader(train_batch_size=512, eval_batch_size=512)

In [12]:
import time
from transformers import Trainer
run = wandb.init(
    # Set the project where this run will be logged
    project="lean3-codet5-finetune-fstar",
    # Track hyperparameters and run metadata
    config={})


data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                mlm=False) # no masked language modelling
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)
trainer.train()


/home/t-sibhat/.local/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:19                                                                                   │
│                                                                                                  │
│   16 │   eval_dataset=test_dataset,                                                              │
│   17 │   data_collator=data_collator,                                                            │
│   18 )                                                                                           │
│ ❱ 19 trainer.train()                                                                             │
│   20                                                                                             │
│                                                                                                  │
│ /home/t-sibhat/.local/lib/python3.9/site-packages/transformers/trainer.py:1645 in train          │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /home/t-sibhat/.local/lib/python3.9/site-packages/transformers/trainer.py:1938 in                │
│ _inner_training_loop                                                                             │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /home/t-sibhat/.local/lib/python3.9/site-packages/transformers/trainer.py:2759 in training_step  │
│                                                                                                  │
│   2756 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2757 │   │                                                                                     │
│   2758 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2759 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2760 │   │                                                                                     │
│   2761 │   │   if self.args.n_gpu > 1:                                                           │
│   2762 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                            

In [ ]:
model.save_pretrained("./output_dir")

In [ ]:
from transformers import T5ForConditionalGeneration

model_gen = T5ForConditionalGeneration.from_pretrained("./output_dir")
# model_gen = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small")
outs_ids = model_gen.generate(tokenizer("for(int i = 0;", return_tensors="pt").input_ids,
    max_length=1024,
    num_beams=4,
    length_penalty=0.0,
    do_sample=False,
    num_return_sequences=4,
    early_stopping=False)
out_str = tokenizer.decode(outs_ids[0], skip_special_tokens=True)
print(out_str)

In [ ]:
# TODO: write evaluation code for definition completion given type?